In [1]:
#Importing Super Mario Bros. video game as well as controlling functionality
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)


from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
import os 
from stable_baselines3 import PPO
from stable_baselines3.common.Images import BaseCallback

#Using matplotlib to display frames from game
from matplotlib import pyplot as plt

In [ ]:
#Initializing the env
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

state = env.reset()

state, reward, done, info = env.step([5])

#Display frames
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1,4,idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

In [ ]:
#Saves current 'image' of model every x iterations automatically, in this case 25000 iterations

class ModelCallbackSave(BaseCallback):

    def __initalize__(self, frequency_check, directory, verbose=1):

        super(ModelCallbackSave, self).__init__(verbose)
        self.frequency_check = frequency_check
        self.directory = directory

    def _init_callback(self):

        if self.directory is not None:
            os.makedirs(self.directory, exist_ok=True)

    def _on_step(self):
        
        if self.n_calls % self.frequency_check == 0:
            model_path = os.path.join(self.directory, 'most_recent_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

DIRECTORY = './models/'

checkpoint = ModelCallbackSave(frequency_check = 25000, directory=DIRECTORY)

In [ ]:
#Running the model

model = PPO.load('./models/most_recent_1000000')

state = env.reset()

while True: 
    
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

